<a href="https://colab.research.google.com/github/Barytes/NBS_project/blob/Plot/plot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import sys

def is_colab():
    return 'google.colab' in sys.modules

if is_colab():
    from google.colab import drive
    drive.mount('/content/drive')
    PROJECT_PATH = '/content/drive/MyDrive/NBS_project/'
else:
    # 自动根据当前脚本定位项目根路径
    PROJECT_PATH = os.path.abspath(os.path.join(os.path.dirname(__file__), '..')) \
        if '__file__' in globals() else os.getcwd()

if PROJECT_PATH not in sys.path:
    sys.path.append(PROJECT_PATH)

print(f"项目路径设置为: {PROJECT_PATH}")

from src.config import Config
from src.main import create_MDs,create_ESP
import src.baseline as baseline
import src.NBS as NBS
import src.model as model
import numpy as np


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
项目路径设置为: /content/drive/MyDrive/NBS_project/


In [ ]:
CONFIG_PATH = os.path.join(PROJECT_PATH, "configs", "test.json")
config = Config(CONFIG_PATH)

{'s': 0.1, 'l': 0.8, 'cn': 0.900369449497976, 'Fn': 1.184383282690447, 'kn': 0.6414529928884467, 'omega_n': 1, 'Rn': np.float64(4.13472934445215)}
{'s': 0.1, 'l': 0.8, 'cn': 1.0415462755795253, 'Fn': 1.7637226511189565, 'kn': 0.6348125738082766, 'omega_n': 1, 'Rn': np.float64(10.248845012378245)}
{'s': 0.1, 'l': 0.8, 'cn': 0.932903940019097, 'Fn': 2.1321345406869066, 'kn': 0.24902581391857953, 'omega_n': 1, 'Rn': np.float64(6.716553285779903)}
{'s': 0.1, 'l': 0.8, 'cn': 0.8278081871246422, 'Fn': 3.8199302863134226, 'kn': 0.35174154718363215, 'omega_n': 1, 'Rn': np.float64(9.231363952354403)}
{'s': 0.1, 'l': 0.8, 'cn': 1.0432905838092195, 'Fn': 3.913825949541608, 'kn': 0.43740643005316393, 'omega_n': 1, 'Rn': np.float64(7.948101841125762)}
{'s': 0.1, 'l': 0.8, 'cn': 0.9766658416311175, 'Fn': 2.4920855793255217, 'kn': 0.5668848317824697, 'omega_n': 1, 'Rn': np.float64(3.859406010253179)}
{'s': 0.1, 'l': 0.8, 'cn': 0.833279172556006, 'Fn': 1.5048957788256199, 'kn': 0.3583132239990442, 'om

In [ ]:
# Experiment 1: Performance of Proposed Algorithm

# Create MDs with a specific seed for reproducibility
seed = 41
MDs = create_MDs(config, seed=seed)
# np.random.shuffle(MDs)  # Shuffle the MDs
    print(md.param)

esp = create_ESP(config, seed=seed)
print(esp.param)

In [ ]:
# Experiment 2: Comparison of NBS and Non-Cooperative

In [ ]:
# Experiment 3: Impact of Bargaining Power

In [ ]:
# Experiment 4: Impact of other parameters

In [5]:
!pwd

/content


In [6]:
%cd '/content/drive/MyDrive/NBS_project'

/content/drive/MyDrive/NBS_project


In [8]:
!git push origin plot

error: src refspec plot does not match any
error: failed to push some refs to 'https://github.com/Barytes/NBS_project.git'
